<a href="https://colab.research.google.com/github/Rumeysakeskin/Turkish-Speech-to-Text/blob/main/transfer_learning_from_English_to_Turkish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# If you're using Google Colab and not running locally, run this cell.
## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install text-unidecode
!pip install matplotlib>=3.3.2

## Install NeMo
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]
!apt-get update && apt-get install -y libsndfile1 ffmpeg
!pip install Cython tensorflow==2.11.0 Pygments==2.6.1 pynini==2.1.5 nemo_toolkit[all]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
sox is already the newest version (14.4.2+git20190427-2).
libsndfile1 is already the newest version (1.0.28-7ubuntu0.1).
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/NVIDIA/NeMo.git (to revision main) to /tmp/pip-install-39fqndvp/nemo-toolkit_c49d86d0e9354254ae6482a21c5920bd
  Running command git clone --filter=blob:none --quiet https://github.com/NVIDIA/NeMo.git /tmp/pip-install-39fqndvp/nemo-toolkit_c49d86d0e9354254ae6482a21c5920bd
  Resolved https://github.com/NVIDIA/NeMo.git to commit 328409

In [ ]:
import nemo.collections.asr as nemo_asr
from ruamel.yaml import YAML
import pytorch_lightning as pl
from omegaconf import DictConfig, OmegaConf, open_dict
import copy
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
import os

if not os.path.exists("configs/config.yaml"):
   !wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/config.yaml

def training_quartznet():


    EPOCHS = 100
   
    config_path = "configs/config.yaml"

    yaml = YAML(typ='safe')
    with open(config_path) as f:
        params = yaml.load(f)

    params['model']['train_ds']['manifest_filepath'] = "scripts/manifest_100522.jsonl" #"data/train_manifest.jsonl"
    params['model']['validation_ds']['manifest_filepath'] = "scripts/manifest_100522.jsonl" #"data/val_manifest.jsonl"


    first_asr_model = nemo_asr.models.EncDecCTCModel.from_pretrained("QuartzNet15x5Base-En")

    
    first_asr_model.change_vocabulary(
        new_vocabulary=[" ", "a", "b", "c", "ç", "d", "e", "f", "g", "ğ", "h", "ı", "i", "j", "k", "l", "m",
                        "n", "o", "ö", "p", "q", "r", "s", "ş", "t", "u", "ü", "v", "w", "x", "y", "z", "'"])

    new_opt = copy.deepcopy(params['model']['optim'])

    new_opt['lr'] = 0.001
    # Point to the data we'll use for fine-tuning as the training set
    first_asr_model.setup_training_data(train_data_config=params['model']['train_ds'])
    # Point to the new validation data for fine-tuning
    first_asr_model.setup_validation_data(val_data_config=params['model']['validation_ds'])
    # assign optimizer config
    first_asr_model.setup_optimization(optim_config=DictConfig(new_opt))


    wandb_logger = WandbLogger(name="Quartznet15x5", project="TURKISH_TRANSFER_LEARNING")
    # used for saving models
    save_path = os.path.join(os.getcwd(),"TURKISH_TRANSFER_LEARNING" + "_" + "Quartznet15x5_models")
    checkpoint_callback = ModelCheckpoint(
        dirpath=save_path,
        save_top_k= -1,
        verbose=True,
        monitor='val_loss',
        mode='min',
    )
    
    trainer = pl.Trainer(gpus=[0], accelerator='cpu',num_nodes=1,
                         max_epochs=EPOCHS,
                         logger=wandb_logger, log_every_n_steps=1,
                         val_check_interval=1.0, enable_checkpointing=checkpoint_callback)

    first_asr_model.set_trainer(trainer)

    trainer.fit(first_asr_model)

if __name__ == '__main__':
    training_quartznet()